# IOS-XE 16.5.1 EFT Image Demo

## Connecting to a Device

Let's define some variables:

In [ ]:
# Local CSR 1000v (running under vagrant) -- rtr1
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

In [ ]:
# Local CSR 1000v (running under vagrant) -- rtr2
#HOST = '127.0.0.1'
#PORT = 2200
#USER = 'vagrant'
#PASS = 'vagrant'

## Test CSR1Kvs in AWS

In [ ]:
HOST = 'ec2-52-90-35-138.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
HOST = 'ec2-54-234-152-160.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
HOST = 'ec2-174-129-180-170.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
HOST = 'ec2-174-129-127-105.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
HOST = 'ec2-52-23-227-161.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
HOST = 'ec2-52-91-252-81.compute-1.amazonaws.com'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

In [ ]:
einar-test1 = ssh cisco@ec2-52-90-35-138.compute-1.amazonaws.com
einar-test2 = ssh cisco@ec2-54-234-152-160.compute-1.amazonaws.com
einar-test3 = ssh cisco@ec2-174-129-180-170.compute-1.amazonaws.com
einar-test4 = ssh cisco@ec2-174-129-127-105.compute-1.amazonaws.com
einar-test5 = ssh cisco@ec2-52-23-227-161.compute-1.amazonaws.com
einar-test6 = ssh cisco@ec2-52-91-252-81.compute-1.amazonaws.com

Now let's establish a NETCONF session to that box using ncclient:

In [ ]:
from ncclient import manager
from lxml import etree

def pretty_print(retval):
        print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [ ]:
for c in m.server_capabilities:
    print(c)

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [ ]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

And now let's look at the capabilities that are related to model support:

In [ ]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

## Schema

Let's take a look at playing with schema. First, we can try downloading them, picking one of the modules we got capabilities for.

In [ ]:
SCHEMA_TO_GET = 'Cisco-IOS-XE-native'

c = m.get_schema(SCHEMA_TO_GET)
print(c.data)

That's not so readable. Let's use a utility called ```pyang``` to get something a bit more readable.

In [ ]:
from subprocess import Popen, PIPE, STDOUT

SCHEMA_TO_GET = 'Cisco-IOS-XE-native'
#SCHEMA_TO_GET = 'ietf-interfaces'


c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]

print(stdout_data)

## What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request

### Getting Config

Let's look at some simple requests...

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

Now let's add in a simple filter:

In [ ]:
filter = '''
<native>
 <username/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Retrieve Interface Data (Native Model)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Retrieve Interface Data (Native Model) With XPath Query

As well as subtree filters, IOS-XE support XPath-based filters.

In [ ]:
# filter = '/native/interface/GigabitEthernet/name'
filter = '/native/interface/*/name'

c = m.get_config(source='running', filter=('xpath', filter))
pretty_print(c)

Let's create an interface, a loopback of course! We'll do it multiple ways.

- IOS-XE Native Model
- IETF Model
- OpenConfig Model

### Native Model

In [ ]:
edit_data = '''<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <Loopback>
        <name>99</name>
        <description>Traditional loopback demo creation</description>
        <ip>
          <address>
            <primary>
              <address>192.168.20.1</address>
              <mask>255.255.255.0</mask>
            </primary>
          </address>
        </ip>
      </Loopback>
    </interface>
  </native>
</config>'''

filter = '''/native/interface/Loopback[name="99"]'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
    c = m.get_config(source='running', filter=('xpath', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### IETF Model

In [ ]:
edit_data = '''<config>
 <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>Loopback100</name>
      <description>Traditional loopback demo creation, IETF model</description>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:softwareLoopback</type>
      <enabled>true</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>192.168.30.1</ip>
          <netmask>255.255.255.0</netmask>
        </address>
      </ipv4>
    </interface>
  </interfaces>
</config>'''

filter = '''<interfaces xmlns="http://openconfig.net/yang/interfaces">
  <interface>
    <name>Loopback100</name>
  </interface>
</interfaces>'''

try:
    #edit_reply = m.edit_config(edit_data, target='running', format='xml')
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### OpenConfig Model

In [ ]:
edit_data = '''<config>
 <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>Loopback100</name>
      <description>Traditional loopback demo creation, IETF model</description>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:softwareLoopback</type>
      <enabled>true</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>192.168.30.1</ip>
          <netmask>255.255.255.0</netmask>
        </address>
      </ipv4>
    </interface>
  </interfaces>
</config>'''

filter = '''<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name>Loopback100</name>
  </interface>
</interfaces>'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Retrieve All BGP Data

Now let's look at the BGP native model:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
 <router>
   <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp"/>
 </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Look At A Specific BGP Neighbor

And can we look at a specific neighbor only? Say the one with id (address) ```192.168.0.1```?

In [ ]:
filter = '''
<native>
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>123</id>
      <neighbor>
        <id>192.168.0.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Create New BGP Neighbor

Ok, so, yes we can get a specific neighbor. Now, can we create a new neighbor? Let's create one with an id of '192.168.1.1', with a remote-as of 666.

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor nc:operation="merge">
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

Now let's pull back that neighbor:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>123</id>
      <neighbor>
        <id>192.168.1.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Modify The BGP Neighbor Description

Can modify something in the neighbor we just created? Let's keep it simple and modify the description:

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor>
          <id>192.168.1.1</id>
          <description nc:operation="merge">modified description</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Delete A BGP Neighbor

Might need to do this before creating depending on the state of the router!

In [ ]:
from ncclient.operations import TimeoutExpiredError
from lxml.etree import XMLSyntaxError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor nc:operation="delete">
          <id>192.168.1.1</id>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except XMLSyntaxError as e:
    print(e)
    print(e.args)
    print(dir(e))
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Other Stuff

Get interface data from native model:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <GigabitEthernet>
      <name/>
      <ip/>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <GigabitEthernet>
      <name>1</name>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Get interfaces from IETF model:

In [ ]:
filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

# Enable Debugging

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)